In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from runner import *
from process_results import *
import os
import pandas as pd
fig_dir = "/usr0/home/aoli/repos/exchain-paper/figs"

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
table_data = []

for name, app in TEST_APPLICATIONS.items():
    table_data.append([])
    table_data[-1].append(name.replace("_", "-").upper())
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()
    root_cause_in_log = check_root_cause_in_log(ground_truth, app.origin_log_path)
    table_data[-1].append(app.is_running_service)
    table_data[-1].append(app.is_async)
    table_data[-1].append(root_cause_in_log)

    table_data[-1].append(get_exception_distance(ground_truth,
                                                   os.path.join(app.get_latest_result("dynamic"), "exception.json")))


    dynamic_result_path = app.get_latest_result("dynamic")
    exceptions = read_exceptions(os.path.join(dynamic_result_path, "exception.json"))
    table_data[-1].append(len(exceptions))
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()


    static_dependency = app.read_latest_static_dependency()
    table_data[-1].extend(process_dependency_result(static_dependency, ground_truth))

    hybrid_online, hybrid_offline = app.read_latest_hybrid_dependency()
    table_data[-1].extend(process_dependency_result(hybrid_online + hybrid_offline, ground_truth))
    table_data[-1].extend(process_dependency_result(dynamic_dependency, ground_truth))
save_as_latex_table(table_data, os.path.join(fig_dir, "issue_result.tex"))

org.apache.wicket.markup.MarkupException:Unable to find component with id 'renderCount' in [Page class = org.apache.wicket.testapplication.TestPage2, id = 1, render count = 1]	Expected: ':renderCount'.	Found with similar names: '':org.apache.wicket.markup.MarkupStream/throwMarkupException:534 -> java.lang.RuntimeException:model still attached!:org.apache.wicket.testapplication.TestPage/onBeforeRender:49

org.apache.wicket.markup.MarkupException:Unable to find component with id 'renderCount' in [Page class = org.apache.wicket.testapplication.TestPage2, id = 1, render count = 1]	Expected: ':renderCount'.	Found with similar names: '':org.apache.wicket.markup.MarkupStream/throwMarkupException:534 -> java.lang.RuntimeException:model still attached!:org.apache.wicket.testapplication.TestPage/onBeforeRender:49



In [11]:

table_data = []

for name, app in BENCH_APPLICATIONS.items():
    separate_results = ["fineract_bench", "jena_bench"]
    if "jena" in name:
        continue
    table_data.append(read_perf_result(app))
save_as_latex_table(table_data, os.path.join(fig_dir, "perf_result.tex"))



(0.17027743824971775, 0.561826233502823)
(0.2429770488773325, 0.09448796799051708)
(0.1970124987219611, 0.34052261507341397)
(0.2050007181825601, 0.27749825257715766)
(0.16243383056722976, 0.6361853769542077)
(0.1323315118488163, 0.8872594050586805)
(0.3309742752356311, 0.0028698627238958893)
(0.20195820278725163, 0.30150267485425714)
(0.20115197153567688, 0.30786356803022497)
(0.10592649062616216, 0.987296018440423)
(0.1661768264947136, 0.6007009439768772)
(0.1465804776358932, 0.7791850248660421)
(0.1673795108177798, 0.589299229392748)
(0.15262727180123792, 0.729153732787823)
(0.1622087741294026, 0.6383189619858012)
(0.3445250811447418, 0.0015797005150113954)
(0.22568527798314308, 0.1706189912279496)
(0.19374269162281987, 0.36632029238130015)
(0.22625347842172935, 0.16799652605141902)
(0.230011194827608, 0.15065320888993772)
(0.160072465423574, 0.6585716431426897)
(0.21259078854037, 0.23105513353991292)
(0.218673564562209, 0.20298076539335258)
(0.40623787903918657, 0.00099999999999988